In [8]:
import cv2.aruco as aruco
import cv2

dictionary = aruco.Dictionary_get(aruco.DICT_4X4_50)
board = aruco.CharucoBoard_create(8, 8, .4, .2, dictionary)

In [60]:
img = board.draw((1024, 1024))

In [67]:
img = cv2.imread("../machines/02_dobot/arucos.png")

In [68]:
params =  aruco.DetectorParameters_create()
corners, ids, rejected = aruco.detectMarkers(
    img,
    dictionary,
    parameters=params
)

num, corners2, ids2 = aruco.interpolateCornersCharuco(
    corners,
    ids,
    img,
    board
)

In [69]:
if img.shape[-1] != 3:
    img_c = np.repeat(img[..., None], 3, axis=-1)
else:
    img_c = img.copy()
img_c = aruco.drawDetectedCornersCharuco(img_c, corners2, ids2, [255, 0, 0])
show(img_c)

In [47]:
mtx = np.array([[6.18050259e+05, 0.00000000e+00, 6.39488488e+02],
        [0.00000000e+00, 6.18050677e+05, 3.59501727e+02],
        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])
dist = np.array([[ 8.07480697e-01,  7.50618764e-07, -8.02591978e-05,
    5.02494853e-05,  5.98330140e-13]])

In [53]:
t = np.zeros((2, 3))
_, rvec, tvec = aruco.estimatePoseCharucoBoard(
    corners2, ids2,
    board, mtx, dist,
    np.zeros(3), np.zeros(3)
)

In [55]:
tvec

array([-1.99843392e+00, -1.12347503e+00,  1.93140716e+03])

In [56]:
cv2.Rodrigues(rvec)

(array([[ 1.00000000e+00, -6.42884228e-08,  9.80081952e-06],
        [ 7.08223995e-08,  9.99999778e-01, -6.66678024e-04],
        [-9.80077448e-06,  6.66678024e-04,  9.99999778e-01]]),
 array([[ 5.34175568e-15,  4.90041333e-06,  3.15997121e-08,
          4.90038331e-06, -6.66678024e-04, -9.99999778e-01,
          3.59556966e-08,  9.99999778e-01, -6.66678024e-04],
        [-9.80079737e-06,  3.33339025e-04,  9.99999926e-01,
          3.33339024e-04,  3.63344779e-13,  3.59556991e-08,
         -9.99999926e-01,  3.15997146e-08, -9.80079700e-06],
        [-6.75554137e-08, -9.99999926e-01,  3.33339024e-04,
          9.99999926e-01, -6.75554112e-08,  4.90041370e-06,
          3.33339025e-04,  4.90038367e-06,  2.50502184e-15]]))

In [54]:
rvec

array([6.66678073e-04, 9.80079773e-06, 6.75554162e-08])

In [36]:
show(img_c)

In [35]:
def show(img):
    cv2.imshow("window", img)
    while True:
        key = cv2.waitKey(1)
        if key == ord('q'):
            break
    cv2.destroyAllWindows()

In [10]:
imboard = 
cv2.imwrite("chessboard1.png", imboard)

True

In [3]:
aruco.Dictionary_get

AttributeError: module 'cv2.aruco' has no attribute 'Dictionary_get'

In [1]:
import numpy as np

In [3]:
np.sign(np.arange(-3, 3))

array([-1, -1, -1,  0,  1,  1])